# Setting up notebook for processing Sentinel-1 ISCE Bursts teken from the ASF Vertex website

In [ ]:
# importing needed libraries for the entire notebook in one go
%matplotlib inline
# %matplotlib widget

import os
from pathlib import Path
from dateutil.parser import parse as parse_date
from datetime import datetime
import pandas as pd
import geopandas as gpd
import numpy as np
from osgeo import gdal, osr
import matplotlib.pyplot as plt
from collections import Counter

import asf_search as asf
import geemap

# Set parameters for search and storage
- edit to include flightDirection and perpendicular baseline?

In [ ]:
def project_dir(cwd, project_name):
    """
    This function reads in a string that you wish to make your working directory 
    for the InSAR project, and creates a data directory to store the data from ASF
    """

    work_dir = os.path.join(cwd, project_name)
    os.mkdir(work_dir)
    data_dir = os.path.join(work_dir,'0_initial')
    os.mkdir(data_dir)

    return work_dir, data_dir

def SLC_search_results(startdate, enddate, wkt_aoi):
    results = asf.search(
        platform= asf.PLATFORM.SENTINEL1A,
        processingLevel=[asf.PRODUCT_TYPE.SLC],
        start = startdate,
        end = enddate,
        intersectsWith = wkt_aoi,
        beamMode= asf.BEAMMODE.IW
    )
    
    return results.geojson(), results

# Set working directory

In [17]:
work_dir, data_dir = project_dir('/home/wcc/Desktop/SabineRS', 'Sentinel-1')

# Draw Area of Interest for your data search

In [19]:
## interactive map for you to draw a polygon to signify your aoi

## Create a map centered at a specific location
m = geemap.Map(center=[20, 0], zoom=2, basemap='HYBRID')

## Add drawing tools
m.add_draw_control()

## Display the map
display(m)

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(child…

In [20]:
# get feature you just drew
draw_features = m.draw_features[0]

# extract the vertices pf the feature
vertices = [(coords[0], coords[1]) for i, coords in enumerate(draw_features.getInfo()['geometry']['coordinates'][0])]

# create POLYGON string to use when searching asf for imagery
aoi = "POLYGON((" + ", ".join([f'{northing} {easting}' for northing, easting in vertices[:]]) +"))"

# Find SLC images for processing
- Backscatter and GLCM used for training classification model
- Phase will be used for subpixel reclassification of the subaqueous land class, resulting in three total classes

In [ ]:
metadata, results = SLC_search_results('2019-10-01', '2024-11-25', aoi)
metadata

# Filter out images along the minority track
- only downloads images that are along the same track, easier interpretation
- remove this step if want all images

In [ ]:

# Extract metadata
features = metadata["features"]
orbits = [feature["properties"]["relativeOrbit"] for feature in features]
frames = [feature["properties"]["frame"] for feature in features]

# Count occurrences of each track and frame
orbit_counts = Counter(orbits)
majority_orbit = orbit_counts.most_common(1)[0][0]

# Filter features by majority track
filtered_features = [f for f in features if f["properties"]["relativeOrbit"] == majority_orbit]

# Optional: Further filter by frame if necessary
frame_counts = Counter([f["properties"]["frame"] for f in filtered_features])
majority_frame = frame_counts.most_common(1)[0][0]
filtered_features = [f for f in filtered_features if f["properties"]["frame"] == majority_frame]

# Create new filtered GeoJSON
filtered_geojson = {
    "type": "FeatureCollection",
    "features": filtered_features
}

# Download SLC images for processing 

In [ ]:
def SLC_download(earthdatausr, earthdatapass, directory, processes):
    session = asf.ASFSession().auth_with_creds(earthdatausr, earthdatapass)

    results.download(
        path = directory,
        session = session, 
        processes = processes 
    )

SLC_download(USERNAME, PASSWORD, data_dir, 2)

# Move to sentinel1prep_snap notebook (not created just yet) for processing using ESA SNAP and snapista python wrapper

In [ ]:
session = asf.ASFSession().auth_with_creds()

results.download(
     path = data_dir,
     session = session, 
     processes = 2 
  )